In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

In [ ]:
pos = pd.read_csv('unique_training_data_queries.csv')
neg = pd.read_csv('random_negative_data.csv')

In [ ]:
neg = neg.drop('General',axis=1)
neg.rename(columns={'Utterance':'input_text'},inplace=True)
neg['target_text'] = 0

In [ ]:
pos.rename(columns={'0':'input_text'}, inplace=True)
pos['target_text'] = 1

In [ ]:
df = pd.concat([neg,pos],axis=0)
df['prefix'] = 'binary classification'

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
train_df = df[:int(len(df)*.75)]
eval_df = df[int(len(df)*.75):]

In [ ]:
train_df.to_csv("data/train.tsv", "\t")
eval_df.to_csv("data/eval.tsv", "\t")

In [ ]:
from simpletransformers.t5 import T5Model

In [ ]:
train_df = pd.read_csv("data/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("data/eval.tsv", sep="\t").astype(str)

In [ ]:
model_args = {
    "max_seq_length": 500,
    "train_batch_size": 16,
    "eval_batch_size": 64,
    "num_train_epochs": 5,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "fp16": False,
    "save_steps": -1,
    "save_eval_checkpoints": True,
    "save_model_every_epoch": False,
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "wandb_project": "T5 mixed tasks - Binary, Multi-Label, Regression",
}

In [ ]:
model = T5Model("t5", "t5-base", args=model_args, use_cuda=False)

In [ ]:
model.train_model(train_df, eval_data=eval_df)

In [ ]:
model.predict('how long is a scale')

In [ ]:
test_df = pd.read_csv('/Users/lioumarc/Downloads/test_dataset (1).csv')

In [ ]:
outputs = []
from tqdm import tqdm
for i in tqdm(list(test_df['query'])):
    outputs.append(model.predict(i))

In [ ]:
len(outputs)

In [ ]:
temp = pd.DataFrame(outputs)

temp

In [ ]:
temp[0].apply(lambda x: x if ((x==1)or(x==0)) else 1)

In [ ]:
test_df['label'] = temp[0].apply(lambda x: x if ((x==1)or(x==0)) else 1)

In [ ]:
test_df[['ID','label']].to_csv('t5-base_trained_on_binary.csv', index=False)